In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import snapshot_download

torch.cuda.is_available()   
torch.zeros(1).cuda()

D:\temp\ipykernel_24972\3262862731.py:6: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  torch.zeros(1).cuda()


tensor([0.], device='cuda:0')

In [5]:
repo_id = 'amgadhasan/phi-2'
# model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-2", local_dir_use_symlinks=False)
model_path = "E:/models/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
torch.set_default_dtype(torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
model.device

device(type='cuda', index=0)

In [9]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :
    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

# prompt = "Write a concise analogy between brain and neural networks"
prompt = "France's capital city is"


In [10]:
result = generate(prompt, generation_params={"max_length":8})
print(result)

Took 3.6 seconds to generate 3 new tokens at speed 0.8 tokens/seconds
France's capital city is Paris.

